In [17]:
from transformers   import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import pandas as pd

In [18]:
data = pd.read_csv("data/immigration_exec_order.csv")
data = data[['text']]
data

,text
0,aka Judge Roberts and 9thCircuit Court in big ...
1,Born and raised in Canada. Not a refugee. Not ...
2,More than 300 protest Trump's immigration ban ...
3,"There is NO #MuslimBan just arrived LAX, no o..."
4,@POTUS @realDonaldTrump https://t.co/9X3WVvFDkl
...,...
239,Somalis in Minnesota: https://t.co/NYPtHAXkvP ...
240,Don't normalize Trump's decision to rewrite im...
241,Four judges have ruled against Trump in the WA...
242,@RT_America @Yaro_RT @ManilaChan #Iran #ISIS ...


In [19]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


In [20]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokeniser = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

In [21]:
#Pretraining
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
text = "Born and raised in Canada. Not a refugee. Not an immigrant. No ties to the 7 counties of the EO. It's a #MuslimBan"
text = preprocess(text)
encoded_input = tokeniser(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

NameError: name 'tokeniser' is not defined

In [14]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1} {l} {np.round(float(s), 4)}")


1 negative 0.4989
2 neutral 0.4823
3 positive 0.0188
